In [27]:
import pandas as pd
import time
import datetime

# Pipeline ETL

In [24]:
def iterate_excel_file_every_5seconds(file_path):
    # Read the Excel file
    xls = pd.read_excel(file_path, header=3)
    for line in xls.iterrows():
        
        print(line)
        time.sleep(5)
        




        break  
    return line  

    

In [41]:
from src.utils.train_utils import load_and_process_data


data_path = '/Users/phamminhtuan/Desktop/VHT_work/MLOps/data/raw/train'
df = load_and_process_data(data_path, exog = True)
df['ds'] = pd.to_datetime(df['ds'], utc = True)
print(df.head())
df.to_parquet("feature_store/feature_repo/data/train_data.parquet", index = False)


       y                        ds unique_id  rolling_mean  rolling_std  \
24  43.0 2025-05-09 02:00:00+00:00  series_1     58.857143    11.596387   
25  43.0 2025-05-09 02:05:00+00:00  series_1     54.857143    11.538755   
26  44.0 2025-05-09 02:10:00+00:00  series_1     51.142857     9.923517   
27  44.0 2025-05-09 02:15:00+00:00  series_1     48.714286     9.159954   
28  42.0 2025-05-09 02:20:00+00:00  series_1     46.142857     7.904188   

    lag_5min  lag_30min  lag_2h  
24      43.0       71.0   103.0  
25      43.0       70.0    96.0  
26      43.0       61.0   100.0  
27      44.0       60.0    89.0  
28      44.0       64.0    82.0  


In [56]:
from feast import FeatureStore
import pandas as pd

# 1️⃣ Entity dataframe (UTC)
entity_df = pd.DataFrame({
    "unique_id": ["series_1"],
    "event_timestamp": ["2025-06-09 00:15:00"]
})
entity_df["event_timestamp"] = pd.to_datetime(entity_df["event_timestamp"])
# 2️⃣ Khởi tạo store
store = FeatureStore(repo_path="feature_store/feature_repo")

# 3️⃣ Lấy historical features
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "interface_traffic:y",
        "interface_traffic:lag_2h",
        "interface_traffic:rolling_mean",
        "interface_traffic:rolling_std",
        "interface_traffic:lag_5min",
        "interface_traffic:lag_30min",
    ],
).to_df()

print(training_df.head())



Empty DataFrame
Columns: [unique_id, event_timestamp, y, lag_2h, rolling_mean, rolling_std, lag_5min, lag_30min]
Index: []


In [54]:
df = pd.read_parquet("data/processed/train_data.parquet")
# print(df.head())
print(df.dtypes)
# print(df["unique_id"].unique())
# print(df["ds"].min(), "→", df["ds"].max())
# print(df.head())
print(entity_df.dtypes)



y                   float64
ds           datetime64[ns]
unique_id            object
dtype: object
unique_id                  object
event_timestamp    datetime64[ns]
dtype: object


In [59]:
import pandas as pd
from feast import FeatureStore

store = FeatureStore(repo_path="feature_store/feature_repo")

# 🔹 Đọc lại source theo đường dẫn trong feature_repo
source_path = "feature_store/feature_repo/data/train_data.parquet"

offline_df = pd.read_parquet(source_path)
print("✅ Loaded source data:")
print("Unique IDs:", offline_df["unique_id"].unique())
print("Time range:", offline_df["ds"].min(), "→", offline_df["ds"].max())
print("dtype:", offline_df["ds"].dtype)


✅ Loaded source data:
Unique IDs: ['series_1']
Time range: 2025-05-09 02:00:00+00:00 → 2025-06-08 23:55:00+00:00
dtype: datetime64[ns, UTC]


In [61]:
import pandas as pd
from feast import FeatureStore

store = FeatureStore(repo_path="feature_store/feature_repo")

# 🔹 Lấy timestamp nằm trong range
entity_df = pd.DataFrame({
    "unique_id": ["series_1"],
    "event_timestamp": pd.to_datetime(["2025-06-07 23:55:00"], utc=True)
})

# 🔹 Lấy historical features
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "interface_traffic:y",
        "interface_traffic:lag_2h",
        "interface_traffic:rolling_mean",
        "interface_traffic:rolling_std",
        "interface_traffic:lag_5min",
        "interface_traffic:lag_30min",
    ],
).to_df()

print(training_df)


  unique_id           event_timestamp      y  lag_2h  rolling_mean  \
0  series_1 2025-06-07 23:55:00+00:00  129.0   324.0    139.428571   

   rolling_std  lag_5min  lag_30min  
0     12.54136     146.0      152.0  


In [ ]:
from feast import FeatureStore
import pandas as pd
import os

def get_training_data(
    repo_path: str,
    feature_view_name: str,
    features: list = None,
    entity_col: str = "unique_id",
    timestamp_col: str = "ds"
) -> pd.DataFrame:
    """
    Lấy toàn bộ historical features từ Feast để huấn luyện mô hình.
    """

    store = FeatureStore(repo_path=repo_path)

    # 🔹 1. Lấy source object và đường dẫn file
    source = store.get_data_source(f"{feature_view_name}_source")
    file_path = 'feature_store/feature_repo/' + source.path  # đường dẫn tới file parquet/csv

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Không tìm thấy file dữ liệu: {file_path}")

    # 🔹 2. Đọc dữ liệu gốc (tùy định dạng)
    if file_path.endswith(".parquet"):
        df_source = pd.read_parquet(file_path)
    elif file_path.endswith(".csv"):
        df_source = pd.read_csv(file_path)
    else:
        raise ValueError("Chỉ hỗ trợ parquet hoặc csv trong ví dụ này.")

    # 🔹 3. Chuẩn hoá thời gian UTC
    df_source[timestamp_col] = pd.to_datetime(df_source[timestamp_col], utc=True)

    # 🔹 4. Tạo entity_df để lấy historical feature
    entity_df = df_source[[entity_col, timestamp_col]].rename(
        columns={timestamp_col: "event_timestamp"}
    )

    # 🔹 5. Nếu chưa truyền danh sách feature, tự động lấy tất cả feature trong view
    if features is None:
        features = [
            f"{feature_view_name}:{f.name}"
            for f in store.get_feature_view(feature_view_name).features
        ]

    # 🔹 6. Lấy historical features từ Feast
    training_df = store.get_historical_features(
        entity_df=entity_df,
        features=features
    ).to_df()

    return training_df
training_df = get_training_data(
    repo_path="feature_store/feature_repo",
    feature_view_name="interface_traffic",
    features=[
        "interface_traffic:y",
        "interface_traffic:lag_2h",
        "interface_traffic:rolling_mean",
        "interface_traffic:rolling_std",
        "interface_traffic:lag_5min",
        "interface_traffic:lag_30min",
    ]
)

print(training_df.head())
print(training_df.shape)


FileNotFoundError: Không tìm thấy file dữ liệu: data/train_data.parquet